In [1]:
import autogen

config_list = autogen.config_list_from_json(
    "C:/Users/denni/OneDrive/Docs16/OAI_CONFIG_LIST5",
    #filter_dict={
    #    "model": ["gpt-4", "gpt-4-1106-preview","microsoft-phi-2","gpt4", "gpt-4-32k", "gpt-4-32k-0314", "gpt-4-32k-v0314"],
    #},
)

C:\Users\denni\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

# 1. create an AssistantAgent instance named "assistant"
coder = autogen.AssistantAgent(
    name="coder", 
    system_message='''You are a helpful assistant. you generate Python code and
    Wolfram Alpha code as well as
    general mathematics. When generating code you prefer Wolfram Alpha. 
    in Wolfram code remember to use * to signify multiplication.
    However, you leave the
    details of execution to mathproxyagent.''',
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }

)

mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent", 
    system_message = '''You are an expert in executing Python 
    and Wolfram Alpha programs.''',
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

In [3]:
critic = autogen.AssistantAgent(
    name="Critic",
    system_message="""Critic. You are a helpful assistant highly skilled in evaluating the 
    quality of a code generated and 
    mathematics produced by the coder. You do not generate code, but only evaluate 
    it and look for errors. leave the code generation 
    to the coder.  in Wolfram code remember to use * to signify 
    multiplication. you leave the
    details of execution to mathproxyagent.
""",
    llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }

)

In [4]:
import os
if "WOLFRAM_ALPHA_APPID" not in os.environ:
    os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()


In [5]:
groupchat = autogen.GroupChat(agents=[mathproxyagent, coder, critic], messages=[], max_round=20)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config={
        "timeout": 600,
        "seed": 42,
        "config_list": config_list,
    })

In [6]:
import os

math_problem = '''Problem: find the point on the parabola (x-3)^2 - (y+5)^2 = 7 that is closest to the
origin.  Try using lagrange multipliers.  and use Wolfram Alpha for all algebraic simplifications. 
'''
mathproxyagent.initiate_chat(manager, problem=math_problem, prompt_type="two_tools")

mathproxyagent (to chat_manager):

Let's use two tools (Python and Wolfram alpha) to solve a math problem.

Query requirements:
You must follow the formats below to write your query:
For Wolfram Alpha:
```wolfram
# one wolfram query
```
For Python:
```python
# your code
```
When using Python, you should always use the 'print' function for the output and use fractions/radical forms instead of decimals. You can use packages like sympy to help you.
When using wolfram, give one query in each code block.

Please follow this process:
1. Solve the problem step by step (do not over-divide the steps).
2. Take out any queries that can be asked through Python or Wolfram Alpha, select the most suitable tool to be used (for example, any calculations or equations that can be calculated).
3. Wait for me to give the results.
4. Continue if you think the result is correct. If the result is invalid or unexpected, please correct your query or reasoning.

After all the queries are run and you get the answ

illustrate a direct call to wolfram alpha

In [10]:
import wolframalpha

# Initialize the client with your AppID
client = wolframalpha.Client('P7xxxxxxxxxPTX')

# Query Wolfram Alpha
res = client.query("solve 2x = 2 lambda (x - 3), 2y = -2 lambda (y + 5), (x - 3)^2 - (y + 5)^2 = 7 for x, y, lambda" )
#print(res)
# Print the result
print(next(res.results).text)

(the variable λ is not present in the given system)


 The constraint is given by the equation of the parabola $(x-3)^2 - (y+5)^2 = 7$, which we can treat as
 $g(x, y) = (x-3)^2 - (y+5)^2 - 7 = 0$.



Using Lagrange multipliers, we set up the following system of equations:
$$
\nabla D^2 = \lambda \nabla g(x, y)
$$
where $\nabla D^2$ is the gradient of the function $D^2$, $\nabla g(x, y)$ is the gradient of the constraint function $g(x, y)$, and $\lambda$ is the Lagrange multiplier.



The gradient of the distance squared function is
$$
\nabla D^2 = \left\langle 2x, 2y \right\rangle
$$
and the gradient of the constraint function is
$$
\nabla g(x, y) = \left\langle 2(x - 3), -2(y + 5) \right\rangle
$$


Setting $\nabla D^2$ equal to \($\lambda \nabla g(x, y)$\), we have two equations:
$$
2x = \lambda \cdot 2(x - 3) \\
2y = \lambda \cdot -2(y + 5)
$$
and our constraint
$$
(x-3)^2 - (y+5)^2 = 7
$$


In [18]:
from autogen import OpenAIWrapper
client = OpenAIWrapper(config_list=config_list)
# ChatCompletion
response = client.create(messages=[{"role": "user", "content": '''what is the difference 
between a parabola and a hyperbola'''}])

In [19]:
def format_clean(s):
    out = ''
    for i in range(len(s)):
        if (s[i] == '\\') and (s[i+1] == 'n'):
            out = out+'/n'
        else:
            out = out+s[i]
    return out

print(format_clean(client.extract_text_or_function_call(response)))  

Both parabola and hyperbola are types of conic sections, which are curves obtained by intersecting a cone with a plane. They have different geometric properties and equations.

Parabola:
1. A parabola is the set of all points in a plane that are equidistant from a fixed point (the focus) and a fixed line (the directrix).
2. The parabola has one axis of symmetry, which passes through the focus and is perpendicular to the directrix.
3. The standard equation of a parabola with its vertex at the origin and axis of symmetry along the x-axis is \(y^2 = 4ax\) or \(x^2 = 4ay\), where \(a\) is the distance from the vertex to the focus.
4. A parabola opens either upwards or downwards (when the axis of symmetry is vertical) or to the right or left (when the axis of symmetry is horizontal), but it does not close on itself; it extends infinitely.
5. Parabolas have one branch.

Hyperbola:
1. A hyperbola, on the other hand, is the set of all points in a plane where the absolute difference in distance